In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


data = pd.read_csv("./data/melb_data.csv")

# Separate target from predictors
y = data.Price
X = data.drop("Price", axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

# find low cardinality
categorical_cols = [cname for cname in X_train_full if X_train_full[cname].nunique() > 10 and X_train_full[cname].dtype == 'object']

# select numerical data
numerical_cols = [cname for cname in X_train_full if X_train_full[cname].dtype in ['int64', 'float64']]

# my selected columns
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)


model = RandomForestRegressor(random_state=0, n_estimators=100)

# Bundle preprocessing and modeling code in bundle
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

my_pipeline.fit(X_train, y_train)


# prediction
preds = my_pipeline.predict(X_valid)
score = mean_absolute_error(y_valid, preds)
print("MAE:", score)

MAE: 167022.08816642122
